## 2. Modelo de Previsão de Precipitação com Machine Learning
### Crie um sistema de previsão de chuva combinando dados meteorológicos multivariados:
### Etapas:
- Use features como pressão atmosférica, umidade, temperatura, velocidade do vento e índices atmosféricos
- Implemente modelos ensemble (Random Forest, XGBoost, LSTM) para previsão de precipitação
- Gere visualizações de importância de features, mapas de probabilidade de chuva e métricas de performance
- Desenvolva relatórios automatizados com acurácia por região e sazonalidade

Este sistema combina técnicas de machine learning com dados meteorológicos multivariados para prever a ocorrência de precipitação. O modelo integra múltiplas fontes de dados e abordagens de modelagem para fornecer previsões precisas e interpretáveis.
Arquitetura do Sistema
1. Coleta e Pré-processamento de Dados

    Fontes de dados: Estações meteorológicas, satélites, radares

    Variáveis incluídas:

        Pressão atmosférica (hPa)

        Umidade relativa (%)

        Temperatura (°C)

        Velocidade e direção do vento (m/s, graus)

        Índices atmosféricos (NAO, SOI, etc.)

        Dados históricos de precipitação

2. Engenharia de Features

    Criação de features temporais (médias móveis, tendências)

    Normalização e padronização dos dados

    Tratamento de valores ausentes e outliers

### Dependencias do projeto

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import contextily as ctx
from shapely.geometry import Point
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam


### Desenvolvimento do modelo

In [9]:
class PrevisaoPrecipitacao:
    def _init__(self, dados_meteorolgicos, dados_geograficos=None):
        """
        Inicializa o sistema de previsão de precipitação.
        
        Args:
            dados_meteorologicos (DataFrame): DataFrame com os dados meteorológicos
            dados_geograficos (GeoDataFrame, optional): Dados geoespaciais das regiões. Defaults to None.
        """
        self.dados = dados_meteorolgicos
        self.dados_geo = dados_geograficos
        self.modelos = {}
        self.ensemble = None
        self.scaler = StandardScaler()
        self.historico_metricas = pd.DataFrame()
    
    def preprocessar_dados(self, target_col='precipitacao', text_size=0.2):
        """
        Pré-processa os dados para modelagem.
        
        Args:
            target_col (str): Nome da coluna alvo. Defaults to 'precipitacao'.
            test_size (float): Proporção para dados de teste. Defaults to 0.2.
        """
        # Limpeza de dados
        self.dados = self.dados.dropna()

        # Separar features e target
        self.X = self.dados.drop(columns=[target_col])
        self.y = self.dados[target_col]
        self.feature_names = self.X. columns.tolist()

        # Normalização
        self.X = pd.DataFrame(self.scaler.fit_transform(self.X), columns=self.feature_names)

        # Split treino-teste
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=text_size, random_state=42
        )
    
    def treinar_modelos(self, timesteps=5):
        """
        Treina os modelos individuais e o ensemble.
        
        Args:
            timesteps (int): Número de passos temporais para LSTM. Defaults to 5.
        """
        # Random Forest
        self.modelos['random_forest'] = RandomForestClassifier(n_estimators=100, random_state=42)
        self.modelos['random_forest'].fit(self.X_train, self.y_train)

        # XGBoost
        self.modelos['xgboost'] = XGBClassifier(learning_rate=0.1, n_estimators=100, random_state=42)
        self.modelos['xgboost'].fit(self.X_train, self.y_train)

        # Preparar dados para LSTM
        X_train_lstm = self._preparar_dados_lstm(self.X_train.values, timesteps)
        X_test_lstm = self._preparar_dados_lstm(self.X_test.values, timesteps)

        # LSTM
        self.modelos['lstm'] = self._construir_modelo_lstm(
            input_shape=(timesteps, self.X_train.shape[1]))
        self.modelos['lstm'].fit(
            X_train_lstm, self.y_train.iloc[timesteps:], 
            epochs=10, batch_size=32, validation_split=0.2, verbose=0
        )

        # Ensemble (sem LSTM por ser sequencial)
        self.ensemble = VotingClassifier(estimators=[
            ('rf', self.modelos['random_forest']),
            ('xgb', self.modelos['xgboost'])
        ], voting='soft')
        self.ensemble.fit(self.X_train, self.y_train)
    
    def _construir_modelo_lstm(self, input_shape):
        """Construi e compila o modelo LSTM"""
        model = Sequential([
            LSTM(64, input_shape=input_shape),
            Dense(1, activation='sigmoid')
        ])
        model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
        return model
    
    def _preparar_dados_lstm(self, dados, timesteps):
        """Prepara os dados no formato sequencial para LSTM."""
        X = []
        for i in range(timesteps, len(dados)):
            X.append(dados[i-timesteps:i, :])
        return np.array(X)
    
    def avaliar_modelos(self):
        """Avalia todos os modelos e armazena as métricas."""
        resultados = {}

        for nome, modelo in self.modelos.items():
            if nome == 'lstm':
                X_test_lstm = self._preparar_dados_lstm(self.X_test.values, 5)
                y_pred = (modelo.predict(X_test_lstm > 0.5)).astype(int)
                y_test = self.y_test.iloc[5:]
            else:
                y_pred = modelo.predict(self.X_test)
                y_test = self.y_test
            
            resultados[nome] = {
                'acuracia': accuracy_score(y_test, y_pred),
                'roc_auc': roc_auc_score(y_test, y_pred) if len(set(y_test)) > 1 else None,
                'relatorio': classification_report(y_test, y_pred, output_dict=True)
            }
        
        # Avaliar ensemble
        y_pred_ensemble = self.ensemble.predict(self.X_test)
        resultados['ensemble'] = {
            'acuracia': accuracy_score(self.y_test, y_pred_ensemble),
            'roc_auc': roc_auc_score(self.y_test, y_pred_ensemble) if len(set(self.y_test)) > 1 else None,
            'relatorio': classification_report(self.y_test, y_pred_ensemble, output_dict=True)
        }
        self.historico_metricas = pd.DataFrame.from_dict(resultados, orient='index')
        return self.historico_metricas


        

